## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import recall_score, precision_score

from src.models.model import GraphClassifier
from src.graphs.graph_loader import GraphDataset
from src.elastic_search_utils.elastic_utils import save_json

## Params

In [2]:
LOAD_FOLDER = '/datasets/johan_tests_original_format_graphs/similarity_shape_100_20__score_threshold_006__similarity_relevance_07/training'

In [3]:
SAVING_FOLDER = '/datasets/johan_tests_models/v1'

In [4]:
SAVING_MODEL_PATH = f'{SAVING_FOLDER}/model.pth'
SAVING_METRICS_PATH = f'{SAVING_FOLDER}/metrics.json'

## Dataset params

In [5]:
BATCH_SIZE = 64

In [6]:
VAL_PERCENTAGE = 0.15
TEST_PERCENTAGE = 0.15

In [7]:
RELEVANCE_THRESHOLD = 0.04  # Keep 20% of elastic most relevant

In [8]:
RANDOM_STATE = 42

In [9]:
torch.manual_seed(RANDOM_STATE)

## Model params

In [10]:
INPUT_DIM = 20
NODES_PER_GRAPH = 100
HIDDEN_CHANNELS = 16
OUTPUT_DIM = 2  # N_CLASSES
DROPOUT = 0.5

In [11]:
DEVICE = torch.device('cuda')

In [12]:
EPOCHS = 100

## Constants

In [13]:
dataset = GraphDataset(
    dataset_path=LOAD_FOLDER,
    batch_size=BATCH_SIZE,
    val_percentage=VAL_PERCENTAGE,
    test_percentage=TEST_PERCENTAGE,
    random_state=RANDOM_STATE,
    score_threshold=RELEVANCE_THRESHOLD
)

In [14]:
model = GraphClassifier(
    input_dims=INPUT_DIM,
    nodes_per_graph=NODES_PER_GRAPH,
    hidden_channels=HIDDEN_CHANNELS,
    output_dim=OUTPUT_DIM,
    dropout=DROPOUT
)

model = model.to(DEVICE)

In [15]:
flat_labels = dataset.metadata['label'].tolist()

class_weights = compute_class_weight(
    'balanced', classes=[0,1], y=flat_labels
)
class_weights = torch.FloatTensor(class_weights)

# MU = 12
# class_weights = torch.FloatTensor([  # GDOT WEIGHTS
#     np.log((MU*7173.0/5886.0) + 1),
#     np.log((MU*7173.0/1287.0) + 1)
# ])
class_weights = class_weights.to(DEVICE)
class_weights

tensor([0.7232, 1.6202], device='cuda:0')

In [16]:
criterion = torch.nn.NLLLoss(
    weight=class_weights
)
criterion = criterion.to(DEVICE)

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-3)

## Training loop

In [18]:
losses = []
accurracies = []
recalls = []
precisions = []
for epoch in range(EPOCHS):
    model.train()
    batch_pbar = tqdm(
        dataset.get_batch('train'),
        desc='loss = inf'
    )

    epoch_loss = 0
    for n_batch, batch in enumerate(batch_pbar):
        batch = batch.to(DEVICE)
        optimizer.step()
        out = model(batch).float()
        batch_weights = torch.where(
            batch.y == 0.0,
            class_weights[0],
            class_weights[1]
        )
        batch_y_test = batch.y #F.one_hot(batch.y, OUTPUT_DIM).float()

        loss = criterion(out, batch_y_test)
        loss.backward()
        optimizer.zero_grad()
        batch.to('cpu')
        batch_pbar.set_description(f"loss = {loss}")
        epoch_loss += loss.to('cpu').tolist()
        # FIXME JUST FOR MEMORY CLEANING
        del out
        del batch_weights
        del batch_y_test
        torch.cuda.empty_cache()
        
    epoch_loss = epoch_loss/(n_batch + 1)
    losses.append(epoch_loss)
    print(f'EPOCH LOSS: {epoch_loss}')
    model.eval()
    y_val = []
    y_pred = []
    batch_pbar_val = tqdm(
        dataset.get_batch('val'),
        desc='loss = inf'
    )
    for test_batch in batch_pbar_val:
        test_batch = test_batch.to(DEVICE)
        out_test = model(test_batch)
        test_gold = F.one_hot(test_batch.y, OUTPUT_DIM).float()
        y_val.append(test_gold.tolist())
        y_pred.append(out_test.tolist())
        test_batch.to('cpu')
        del out_test
        del test_gold
        torch.cuda.empty_cache()
    y_val = np.concatenate(y_val)
    y_pred = np.concatenate(y_pred)

    correct = (y_val.argmax(axis=1) == y_pred.argmax(axis=1)).sum().item()
    acc = int(correct) / len(y_val)
    recall = recall_score(
        y_val.argmax(axis=1), y_pred.argmax(axis=1), average='binary'
    )
    precision = precision_score(
        y_val.argmax(axis=1), y_pred.argmax(axis=1), average='binary'
    )
    accurracies.append(acc)
    recalls.append(recall)
    precisions.append(precision)
    print("Value counts", pd.value_counts(np.array(y_pred.argmax(axis=1))))
    print("Recall", recall)
    print("Precision", precision)
    print(f'Accuracy: {acc:.4f}')

loss = 0.6931470632553101: : 4290it [11:01,  6.48it/s]


EPOCH LOSS: 0.6932861921253738


loss = inf: 908it [01:55,  7.86it/s]
/datasets/anaconda3/envs/torch1.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Value counts 0    58068
dtype: int64
Recall 0.0
Precision 0.0
Accuracy: 0.7018


loss = 0.6931470632553101: : 4290it [11:30,  6.22it/s]


EPOCH LOSS: 0.6931470574198902


loss = inf: 908it [01:58,  7.64it/s]
/datasets/anaconda3/envs/torch1.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Value counts 0    58068
dtype: int64
Recall 0.0
Precision 0.0
Accuracy: 0.7018


loss = 0.6931470632553101: : 239it [00:39,  6.06it/s]


KeyboardInterrupt: 

In [19]:
time_series = {
    'loss': losses,
    'accuracy': accurracies,
    'recall': recalls,
    'precision': precisions
}

In [20]:
save_json(time_series, SAVING_METRICS_PATH)

In [21]:
torch.save(model.state_dict(), SAVING_MODEL_PATH)

## Test load model

In [22]:
model = GraphClassifier(
    input_dims=INPUT_DIM,
    nodes_per_graph=NODES_PER_GRAPH,
    hidden_channels=HIDDEN_CHANNELS,
    output_dim=OUTPUT_DIM,
    dropout=DROPOUT
)

model = model.to(DEVICE)

In [23]:
model.load_state_dict(torch.load(SAVING_MODEL_PATH))

<All keys matched successfully>

In [30]:
model.eval()
y_val = []
y_pred = []
batch_pbar_val = tqdm(
    dataset.get_batch('val'),
    desc='loss = inf'
)
for test_batch in batch_pbar_val:
    test_batch = test_batch.to(DEVICE)
    out_test = model(test_batch)
    test_gold = F.one_hot(test_batch.y, OUTPUT_DIM).float()
    y_val.append(test_gold.tolist())
    y_pred.append(out_test.tolist())
    test_batch.to('cpu')
    del out_test
    del test_gold
    torch.cuda.empty_cache()
y_val = np.concatenate(y_val)
y_pred = np.concatenate(y_pred)

correct = (y_val.argmax(axis=1) == y_pred.argmax(axis=1)).sum().item()
acc = int(correct) / len(y_val)
recall = recall_score(
    y_val.argmax(axis=1), y_pred.argmax(axis=1), average='binary'
)
precision = precision_score(
    y_val.argmax(axis=1), y_pred.argmax(axis=1), average='binary'
)
accurracies.append(acc)
recalls.append(recall)
precisions.append(precision)
print("Value counts", pd.value_counts(np.array(y_pred.argmax(axis=1))))
print("Recall", recall)
print("Precision", precision)
print(f'Accuracy: {acc:.4f}')

loss = inf: 908it [01:16, 11.88it/s]

Value counts 0    58068
dtype: int64
Recall 0.0
Precision 0.0
Accuracy: 0.7018



/datasets/anaconda3/envs/torch1.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
